<a href="https://colab.research.google.com/github/LeoCh97/Tesi/blob/master/Image_loader_balanced_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as matplot
import os 
import cv2
from tqdm import tqdm_notebook 
import pandas as pd
from imblearn import over_sampling

DATADIR1 = "FlickrLogos-v2/classes/jpg"
DATADIR2 = "images"
BBDIR1 = "FlickrLogos-v2/classes/masks"
IMG_SIZE = 128

with open("Flickrplus_num.txt", 'r') as f:
  if f.mode == 'r':
    NUM_OF_IMG = int(f.read())

CATEGORIES = []
for category in os.listdir(DATADIR1):
    if category != ".DS_Store":
        CATEGORIES.append(category)
try:
  assert len(CATEGORIES) == 33
except AssertionError as e:
  print("Wrong number of classes", e)

In [0]:
print(CATEGORIES)

In [0]:
train_data_names = pd.read_csv('FlickrLogos-v2/trainset.txt', header=None, names=['category', 'img_name'])
test_data_names = pd.read_csv('FlickrLogos-v2/testset.txt', header=None, names=['category', 'img_name'])
val_data_names = pd.read_csv('FlickrLogos-v2/valset.txt', header=None, names=['category', 'img_name'])
data_names_plus = pd.read_csv('Flickrplus_groundtruth.txt')

data_names_plus = data_names_plus.values.tolist()
train_data_names = train_data_names.values.tolist()
test_data_names = test_data_names.values.tolist()
val_data_names = val_data_names.values.tolist()

In [0]:
train_split = []

def create_train_data():
  for category,img in tqdm_notebook(train_data_names):
    if category == "guiness":
      data_path = os.path.join(DATADIR1, "guinness")
      bbox_path = os.path.join(BBDIR1, "guinness")
      data_num = CATEGORIES.index("guinness")
    else:
      data_path = os.path.join(DATADIR1, category)
      bbox_path = os.path.join(BBDIR1, category)
      data_num = CATEGORIES.index(category)
    
    try:
      img_array = cv2.imread(os.path.join(data_path,img))
      img_bbox = pd.read_csv(os.path.join(bbox_path,img+".bboxes.txt"), sep=" ")
      
      x = img_bbox['x'][0]
      y = img_bbox['y'][0]
      w = img_bbox['width'][0]
      h = img_bbox['height'][0]
      
      img_array = img_array[y:y+h, x:x+w]
      new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
      train_split.append([new_array, data_num])
    except Exception as e:
      print("general exception", e, os.path.join(data_path,img))
    except OSError as e:
      print("OSError Bad img most likely", e, os.path.join(data_path,img)) 
    
create_train_data()

In [0]:
test_split = []

def create_test_data():
  for category,img in tqdm_notebook(test_data_names):
    if category == "guiness":
      data_path = os.path.join(DATADIR1, "guinness")
      bbox_path = os.path.join(BBDIR1, "guinness")
      data_num = CATEGORIES.index("guinness")
    else:
      data_path = os.path.join(DATADIR1, category)
      bbox_path = os.path.join(BBDIR1, category)
      data_num = CATEGORIES.index(category)
    
    try:
      img_array = cv2.imread(os.path.join(data_path,img))
      
      if category != "no-logo":
        img_bbox = pd.read_csv(os.path.join(bbox_path,img+".bboxes.txt"), sep=" ")
      
        x = img_bbox['x'][0]
        y = img_bbox['y'][0]
        w = img_bbox['width'][0]
        h = img_bbox['height'][0]
      
        img_array = img_array[y:y+h, x:x+w]
      
      new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
      test_split.append([new_array, data_num])
    except Exception as e:
      print("general exception", e, os.path.join(data_path,img))
    except OSError as e:
      print("OSError Bad img most likely", e, os.path.join(data_path,img)) 
    
  
create_test_data()

In [0]:
val_split = []

def create_val_data():
  for category,img in tqdm_notebook(val_data_names):
    if category == "guiness":
      data_path = os.path.join(DATADIR1, "guinness")
      bbox_path = os.path.join(BBDIR1, "guinness")
      data_num = CATEGORIES.index("guinness")
    else:
      data_path = os.path.join(DATADIR1, category)
      bbox_path = os.path.join(BBDIR1, category)
      data_num = CATEGORIES.index(category)
    
    try:
      img_array = cv2.imread(os.path.join(data_path,img))
      
      if category != "no-logo":
        img_bbox = pd.read_csv(os.path.join(bbox_path,img+".bboxes.txt"), sep=" ")
      
        x = img_bbox['x'][0]
        y = img_bbox['y'][0]
        w = img_bbox['width'][0]
        h = img_bbox['height'][0]
      
        img_array = img_array[y:y+h, x:x+w]
      
      new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
      val_split.append([new_array, data_num])
    except Exception as e:
      print("general exception", e, os.path.join(data_path,img))
    except OSError as e:
      print("OSError Bad img most likely", e, os.path.join(data_path,img)) 
    
  
create_val_data()

In [0]:
print(data_names_plus[NUM_OF_IMG-1][1])

In [0]:
def extend_splits():
  racoons = 0
  (train_elements, val_elements, cat) = (0, 0, data_names_plus[0][1])
  
  for filename, category, x, y, w, h, bbox in tqdm_notebook(data_names_plus[0:NUM_OF_IMG]):
    img_path = os.path.join(DATADIR2, filename)
    data_num = CATEGORIES.index(category)
    
    try:
      img = cv2.imread(img_path)
      new_array = img[y:y+h, x:x+w]
      
      new_array = cv2.resize(new_array, (IMG_SIZE, IMG_SIZE))
      
      if cat == category:
        if train_elements < 80:
          train_split.append([new_array, data_num])
          train_elements += 1
        elif val_elements < 10:
          val_split.append([new_array, data_num])
          val_elements += 1
        else:
          test_split.append([new_array, data_num])
      else:
        (train_elements, val_elements, cat) = (1, 0, category)
        train_split.append([new_array, data_num])
    except Exception as e:
      print("general exception", e, img_path)
    except OSError as e:
      print("OSErrorBad img most likely", e, img_path) 
        
    try:
      for giraffe in range(2, bbox):
        x_1 = data_names_plus[NUM_OF_IMG-1+giraffe+racoons][2]
        y_1 = data_names_plus[NUM_OF_IMG-1+giraffe+racoons][3]
        w_1 = data_names_plus[NUM_OF_IMG-1+giraffe+racoons][4]
        h_1 = data_names_plus[NUM_OF_IMG-1+giraffe+racoons][5]
        new_array = img[y:y+h, x:x+w]

        new_array = cv2.resize(new_array, (IMG_SIZE, IMG_SIZE))

        if cat == category:
          if train_elements < 80:
            train_split.append([new_array, data_num])
            train_elements += 1
          elif val_elements < 10:
            val_split.append([new_array, data_num])
            val_elements += 1
          else:
            test_split.append([new_array, data_num])
        else:
          (train_elements, val_elements, cat) = (1, 1, category)
          train_split.append([new_array, data_num])

      racoons += bbox - 1

    except Exception as e:
      print("general exception 2", e, img_path)
    except OSError as e:
      print("OSErrorBad img most likely 2", e, img_path) 
    
    
extend_splits()

In [0]:
assert len(train_split) == 32*90
assert len(val_split) == 32*40+3000

In [0]:
print(len(train_split))
print(len(test_split))
print(len(val_split))

In [0]:
for n in range(len(CATEGORIES)):
  index = 0
  for sample in train_split:
    if sample[1] == n:
      index += 1

  print(index)

In [0]:
import random

random.shuffle(train_split)
random.shuffle(test_split)
random.shuffle(val_split)

In [0]:
for sample in train_split[:10]:
    print(sample[1])

In [0]:
X_train = []
y_train = []

for features,label in train_split:
    X_train.append(features)
    y_train.append(label)

X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [0]:
X_test = []
y_test = []

for features,label in test_split:
    X_test.append(features)
    y_test.append(label)

X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [0]:
X_val = []
y_val = []

for features,label in val_split:
    X_val.append(features)
    y_val.append(label)

X_val = np.array(X_val).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [0]:
import pickle

pickle_out = open("X_train.pickle","wb")
pickle.dump(X_train, pickle_out)
pickle_out.close()

pickle_out = open("y_train.pickle","wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

pickle_out = open("X_test.pickle","wb")
pickle.dump(X_test, pickle_out)
pickle_out.close()

pickle_out = open("y_test.pickle","wb")
pickle.dump(y_test, pickle_out)
pickle_out.close()

pickle_out = open("X_val.pickle","wb")
pickle.dump(X_val, pickle_out)
pickle_out.close()

pickle_out = open("y_val.pickle","wb")
pickle.dump(y_val, pickle_out)
pickle_out.close()